# Imports

In [3]:
import pandas as pd
import numpy as np
import math

# Load in datasets

In [4]:
# Load in dataset
conversions = pd.read_csv("../../Data/Conversions.csv").astype('string')

## Clean moves

In [5]:
# Load moves
moves = pd.read_csv("../../Data/moves.csv", sep=";")

# convert years to ints.
moves['Perioden'] = moves['Perioden'].map(lambda x: x.rstrip('00').rstrip("JJ"))
moves['Perioden'] = pd.to_datetime(moves['Perioden'], format='%Y').dt.year
moves.reset_index(inplace=True)
moves.drop(columns = ['index'],inplace=True)

In [6]:
# Load in top 10 townships
top_10_townships = ['GM0363','GM0599','GM0518','GM0344','GM0772','GM0855','GM0014','GM0034','GM0758','GM0268']

# Only keep rows where at least the vestiging or vertrek is one of the top 10 cities
moves = moves.loc[(moves['RegioVanVestiging'].isin(top_10_townships)) | (moves['RegioVanVertrek'].isin(top_10_townships))]

In [7]:
# Drop ten boer, cause fuck em
ten_boer = "GM0009"
tb1 = moves.loc[(moves['RegioVanVestiging']== "GM0014") & (moves['RegioVanVertrek'] == "GM0009")].index
tb2 = moves.loc[(moves['RegioVanVestiging']== "GM0009") & (moves['RegioVanVertrek'] == "GM0014")].index
moves.drop(tb1, inplace=True)
moves.drop(tb2, inplace=True)

In [8]:
# Convert all municipalty codes to the new ones.
conv_dict = pd.Series(conversions['newCode'].values, index = conversions['oldCode']).to_dict()
moves['RegioVanVestiging'] = moves['RegioVanVestiging'].apply(lambda x: conv_dict[x] if x in conv_dict.keys() else x)
moves['RegioVanVertrek'] = moves['RegioVanVertrek'].apply(lambda x: conv_dict[x] if x in conv_dict.keys() else x)

moves = moves.groupby(['RegioVanVestiging', 'RegioVanVertrek', 'Perioden']).agg({'TussenGemeentenVerhuisdePersonen_1': 'sum'}).reset_index()

In [9]:
# Drop 4 municipalites that were converted before 2015, but still poison the dataset.
todrop = {'GM0366', 'GM0412', 'GM0462', 'GM0463'}
idx = moves[moves["RegioVanVestiging"].isin(todrop) | moves["RegioVanVertrek"].isin(todrop)].index
moves.drop(idx, inplace=True)

In [10]:
# Drop all rows where municipality being moved to is the same as being moved from
idx = moves[(moves['RegioVanVertrek'] == moves['RegioVanVestiging'])].index
moves.drop(idx, inplace=True)

In [11]:
# Save the dataset
moves.to_pickle("../../cleanedDFs/moves.pkl")

## Clean prices

In [12]:
# Load prices
prices = pd.read_csv("../../Data/house_prices_new.csv", sep = ';')

In [13]:
# Drop everything that is not a municipality
prices = prices.loc[285:]
prices.reset_index(inplace=True)
prices.drop(columns = 'index',inplace=True)

# Filter the year code to become int64
prices['Perioden'] = prices['Perioden'].map(lambda x: x.rstrip('00').rstrip("JJ"))
prices['Perioden'] = pd.to_datetime(prices['Perioden'], format='%Y').dt.year

# Replace "." with nan values and convert to float
prices.loc[ prices['GemiddeldeWoningwaarde_4'] == '       .', 'GemiddeldeWoningwaarde_4'] = np.nan
prices['GemiddeldeWoningwaarde_4'] = prices['GemiddeldeWoningwaarde_4'].astype(float)

In [14]:
# Convert all municipalty codes to the new ones.
conv_dict = pd.Series(conversions['newCode'].values, index = conversions['oldCode']).to_dict()
prices['RegioS'] = prices['RegioS'].apply(lambda x: conv_dict[x] if x in conv_dict.keys() else x)

# Group by regio and year and add up woningen_2 and take the mean of gemiddeldewoningwaarde_4.
prices = prices.groupby(['RegioS', 'Perioden']).agg({'Woningen_2':'sum','GemiddeldeWoningwaarde_4': 'mean'}).reset_index()

In [15]:
# Remove all municipalities from the prices df that are not in the moves df (RegiovanVertrek & RegioVanVestiging are the same size.)
unique_prices = prices.RegioS.unique()
unique_moves = moves.RegioVanVestiging.unique()
difference = set(unique_prices) - set(unique_moves)
difference2 = set(unique_moves) -set(unique_prices) # There is no municipality in prices that is not in moves.
difference_index = prices.loc[prices["RegioS"].isin(difference)].index
prices.drop(difference_index, inplace=True)

In [16]:
# Save the dataset
prices.to_pickle("../../cleanedDFs/prices.pkl")

## Clean population

In [17]:
# Load in the dataset
population = pd.read_csv("../../Data/population_size.csv", sep = ';')

# Do the thing with the gender that Bram did
population = population[population['Geslacht'] == 'T001038']

#Drop SaldoAdministratieveCorrecties as is it of no informative use to us
population.drop(['SaldoAdministratieveCorrecties_15'], axis=1, inplace=True)

# Filter the year code to become int64
population['Perioden'] = population['Perioden'].map(lambda x: x.rstrip('00').rstrip("JJ"))
population['Perioden'] = pd.to_datetime(population['Perioden'], format='%Y').dt.year

In [18]:
# Holy shit het klopt (Dit is een test om te kijken of de CBS data tussen datasets overeenkomt)
headers = population.columns.values.tolist()

# Dit is de som van alle mensen die uit amsterdam zijn vertrokken
dim = moves[moves["RegioVanVestiging"] == "GM0363"]
dim2 = dim[dim["Perioden"] == 2017]
print("People who moved from amsterdam according to moves dataset: ", sum(dim2["TussenGemeentenVerhuisdePersonen_1"]))

dom = population[population["RegioS"]=="GM0363"]
dom2 = dom[dom["Perioden"] == 2017]
print("People who moved from amsterdam according to population dataset: ", dom2["UitAndereGemeente_11"])

People who moved from amsterdam according to moves dataset:  36285.0
People who moved from amsterdam according to population dataset:  186    36285.0
Name: UitAndereGemeente_11, dtype: float64


In [19]:
# Make a dictionary of the relevant headers and their aggregation method

header_agg = { 'BevolkingOp1Januari_1': 'sum', 
  'Bevolkingsdichtheid_2': 'sum', 
  'LevendGeborenKinderen_3': 'sum', 
  'LevendGeborenKinderenRelatief_4' : 'mean',   
  'Overledenen_5': 'sum', 
  'OverledenenRelatief_6': 'mean', 
  'Geboorteoverschot_7': 'sum', 
  'TotaalVestiging_8': 'sum', 
  'TotaalVestigingRelatief_9': 'mean', 
  'Immigratie_10': 'sum', 
  'UitAndereGemeente_11': 'sum', 
  'TotaalVertrekInclusiefCorrecties_12': 'sum', 
  'TotaalVertrekInclusiefCoRelatief_13': 'mean', 
  'EmigratieInclusiefAdministratieveC_14': 'sum',
  'NaarAndereGemeente_16': 'sum', 
  'VestigingsoverschotInclusiefCorrecties_17': 'sum', 
  'TotaleGroei_18': 'sum', 
  'TotaleGroeiRelatief_19': 'mean', 
  'BevolkingOp31December_20': 'sum'
}

# Convert all object items to floats to avoid errors:
population['LevendGeborenKinderenRelatief_4'] = population['LevendGeborenKinderenRelatief_4'].astype(float)
population['OverledenenRelatief_6'] = population['OverledenenRelatief_6'].astype(float)
population['TotaalVestigingRelatief_9'] = population['TotaalVestigingRelatief_9'].astype(float)
population['TotaleGroeiRelatief_19'] = population['TotaleGroeiRelatief_19'].astype(float)
population['TotaalVertrekInclusiefCoRelatief_13'] = population['TotaalVertrekInclusiefCoRelatief_13'].astype(float)
population['BevolkingOp1Januari_1'] = population['BevolkingOp1Januari_1'].astype(float)
population['Bevolkingsdichtheid_2'] = population['Bevolkingsdichtheid_2'].astype(float)


In [20]:
# Convert all municipalty codes to the new ones.
conv_dict = pd.Series(conversions['newCode'].values, index = conversions['oldCode']).to_dict()
population['RegioS'] = population['RegioS'].apply(lambda x: conv_dict[x] if x in conv_dict.keys() else x)

# Group by regio and year and use the aggregation dictionary to decide on the aggregation per column.
population = population.groupby(['RegioS', 'Perioden']).agg(header_agg).reset_index()

In [21]:
# Remove all municipalities from the population df that are not in the moves df (RegiovanVertrek & RegioVanVestiging are the same size.)
unique_population = population.RegioS.unique()
unique_moves = moves.RegioVanVestiging.unique()
difference = set(unique_population) - set(unique_moves)
difference2 = set(unique_moves) -set(unique_population) # There is no municipality in population that is not in moves.
difference_index2 = population.loc[population["RegioS"].isin(difference)].index
population.drop(difference_index2, inplace=True)

In [22]:
population.to_pickle("../../cleanedDFs/population.pkl")

## Clean availability

In [23]:
# Load the dataset
availy = pd.read_csv("../../Data/house_availability.csv", sep = ';')

# Filter the year code to become int64
availy['Perioden'] = availy['Perioden'].map(lambda x: x.rstrip('00').rstrip("JJ"))
availy['Perioden'] = pd.to_datetime(availy['Perioden'], format='%Y').dt.year

# Drop all 2021 periods
availy = availy.drop(availy[availy.Perioden == 2021].index)


In [24]:
column_headers = availy.columns.values.tolist()
column_headers
header_dict = {
 'TotaleWoningvoorraad_1': 'sum',
 'Koopwoningen_2': 'sum',
 'TotaalHuurwoningen_3': 'sum',
 'EigendomWoningcorporatie_4': 'sum',
 'EigendomOverigeVerhuurders_5': 'sum',
 'EigendomOnbekend_6': 'sum'
}

In [25]:
# Convert all municipalty codes to the new ones.
conv_dict = pd.Series(conversions['newCode'].values, index = conversions['oldCode']).to_dict()
availy['RegioS'] = availy['RegioS'].apply(lambda x: conv_dict[x] if x in conv_dict.keys() else x)

# Group by regio and year and use the aggregation dictionary to decide on the aggregation per column.
availy = availy.groupby(['StatusVanBewoning', 'RegioS', 'Perioden']).agg(header_dict).reset_index()

In [26]:
# Remove all municipalities from the prices df that are not in the moves df (RegiovanVertrek & RegioVanVestiging are the same size.)
unique_availy = availy.RegioS.unique()
unique_moves = moves.RegioVanVestiging.unique()
difference = set(unique_availy) - set(unique_moves)
difference3 = set(unique_moves) -set(unique_availy) 
difference_index3 = availy.loc[availy["RegioS"].isin(difference)].index
availy.drop(difference_index3, inplace=True)

There are three types of "StatusVanBewoning":

    A028725 = Bewoonde woning.
    A028726 = Onbewoonde woning.
    T001235 = Totale voorraad woningen.
    
It holds for all regions for a specific year that: A028725 + A028726 = T001235 For all values in the columns. We only retain 'T001235', as this gives the most complete picture.

In [27]:
availy = availy.loc[availy['StatusVanBewoning'] == 'T001235']
availy.reset_index(inplace=True)
availy.drop(columns='index',inplace=True)

In [28]:
availy.to_pickle("../../cleanedDFs/availability.pkl")

# Endnotes/TODO's 

-Er is overlap tussen kolom "Woningen_2" van de prices tabel en "TotaleWoningvoorraad_1" van de availability dataset. De cijfers komen echter niet overeen, wat enigszins raar is gezien andere overlappende waardes in CBS tabellen wel overeenkomen.
(We laten ze staan voor nu)


-Alle gecleanede datasets zijn nu opgeslagen in de map cleanedDFs.